
# Einstein Wave Propagation

## The Einstein's wave Equation



### Problem (1):
Given a symmetric field $\gamma_0$ and $\gamma_1$ satisfying the equations:
$$
\begin{align*}
\begin{cases}
     \text{div }  \text{div }  S\gamma_0 =0\\
    \text{div } S\gamma_1 =0
\end{cases}
\end{align*}
$$
Find symmetric matrix fields $\gamma(t)$ Such that
$$
\begin{cases}
\gamma(0)  = \gamma_0 \\
\gamma'(0) = \gamma_1
\end{cases}
$$
$$
\begin{align*}
    S\gamma'' = - \text{inc }\gamma \\
\end{align*}
$$



Using the change of variable $S\gamma' = \kappa$ and $S\gamma_1 = \kappa_0$


### Porblem (2)

Given a symmetric field $\gamma_0$ and $\kappa_0$ satisfying the equations:
$$
\begin{align*}
\begin{cases}
     \text{div }  \text{div }  S\gamma_0 =0\\
    \text{div } \kappa_0 =0
\end{cases}
\end{align*}
$$

Find symmetric matrix fields $\gamma(t)$ such that

$$
\begin{align*}
    \kappa' &= - \text{inc }\gamma \\
    \gamma' &= J\kappa
\end{align*}
$$

We need to modify it into a Discrete Variational form, now we don't care what the initial conditions are. 


### Porblem (3)
For each time step $n$ we consider the following procedure:

1. 
$$
\begin{cases}
\hat\kappa^{n}  = \text{inc }\gamma^{n} \\
\hat\gamma^{n}  = \text{J }\kappa^{n} 
\end{cases}
$$

2. 
$$
\begin{cases}
\gamma^{n+1}  = \gamma^{n} + \tau \hat\gamma^{n} 
\end{cases}
$$

3.
$$
\begin{cases}
\kappa^{n+1}  = \kappa^{n} - \tau \hat\kappa^{n} \\
\text{div} \kappa^{n+1} = 0 
\end{cases}
$$


The stepping foreward will reduce to : 

\begin{align}
\begin{cases}
        \begin{pmatrix}
            \psi^{n+1}\\
            p 
        \end{pmatrix}
        =
        \begin{pmatrix}
            M_{\kappa \kappa } &  (M^{\text{div}}_{\kappa c})^T \\
            M^{\text{div}}_{\kappa c} &      -C 
        \end{pmatrix}^{-1}
        \begin{pmatrix}
            M_{\kappa\kappa} & 0 \\ 0 & M_{c c}
        \end{pmatrix}
        \begin{pmatrix}
            \kappa^n -\tau (M_{\kappa\kappa})^{-1}M_{\kappa\gamma}(M_{\gamma\gamma})^{-1} M^{\text{inc}}_{\gamma\gamma} \gamma^{n}\\
            0
        \end{pmatrix}
        \\
        \gamma^{n+1} = \gamma^n+ \tau (M_{\gamma \gamma})^{-1}M_{\gamma\kappa}( M_{\kappa\kappa})^{-1} M^J_{\kappa\kappa} \kappa^{n}
    \end{cases}
\end{align}

With $C = 10^{-9}(\left\langle p_i,q_j \right\rangle)_{ij}$ Gram matrix of the basis functions of $H(\text{curl})$.




In [1]:
from ngsolve import *
from netgen.occ import *
import matplotlib.pyplot as plt
from ngsolve.webgui import Draw

import time
import scipy.sparse as sp
from scipy.sparse import csc_matrix


In [2]:
def YoshidaIntegratorStep(tau,u,Mu,v,Mv):
    # https://en.wikipedia.org/wiki/Leapfrog_integration#cite_note-Yoshida1990-6
    # book yoshida symplectic
    # convention Mv is the multiplication matrix for v
    # convention Mu is the multiplication matrix for u
    w0 = - 2**-(1/3)/(2-2**-(1/3))
    w1 = 1/(2-2**-(1/3))

    c= [w0/2 , (w0+w1)/2  , (w0+w1)/2 , w0/2 ]
    d= [w1,w0,w1]

    u.vec.data += c[0]*tau*Mv*v.vec
    v.vec.data += d[0]*tau*Mu*u.vec

    u.vec.data += c[1]*tau*Mv*v.vec
    v.vec.data += d[1]*tau*Mu*u.vec

    u.vec.data += c[2]*tau*Mv*v.vec
    v.vec.data += d[2]*tau*Mu*u.vec 

    u.vec.data += c[3]*tau*Mv*v.vec
    return u,v

In [3]:
def norm(u, Mesh):
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh))
        


In [4]:
def Curl(u):
    if u.dim == 3:
        return CF( (u[2].Diff(y)- u[1].Diff(z), u[0].Diff(z)- u[2].Diff(x), u[1].Diff(x)- u[0].Diff(y)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )


In [5]:

def Inc(u):
    return Curl(Curl(u).trans)


In [6]:

def Div(u):
    if u.dim == 3:
        return CF( (u[0].Diff(x)+ u[1].Diff(y) + u[2].Diff(z)) )
    if u.dim == 9:
        return CF( (Div(u[0,:]),Div(u[1,:]),Div(u[2,:])) )


In [7]:

def Tr(u):
    return (u[0,0]+u[1,1]+u[2,2])


In [8]:

def TensorCross(A,B): 
    return 0.5*Cof(A+B)-0.5*Cof(A-B)

def J(u) : return u - 0.5*Id(3)
def SymCurl(u) : return (Curl(u)+Curl(u).trans)/2

In [9]:


def EinsteinPropagation(mesh , order = 1 , initail_g_field = None , bonus_intorder = 10 ,draw_g_field = False , draw_k_field = False, draw_divk_field = False , final_time  = 0.1, step_time = 0.002 ,video = False):
    

    print("Create FESpaces H(cc)XH(dd)")
    Hcc = HCurlCurl(mesh, order=order)
    Hdd = HDivDiv(mesh, order=order   , dirichlet= ".*")
    Hc = HCurl(mesh,  order=order-1)
    fes = Hcc*Hdd
    print("Create projection FES H(dd)XH(c)")
    fes_projection =  Hdd*Hc

    print("Create GridFunction for H(cc)XH(dd) and initialize the g field")
    weak_field = GridFunction(fes) 
    gf_g, gf_k = weak_field.components
    gf_k.Set ( initail_g_field, bonus_intorder=bonus_intorder)#, dual=True)


    # projectional part
    print("Create projection GridFunction for H(dd)XH(c)")
    projection_field = GridFunction(fes_projection) 
    Pk, Pq = projection_field.components

    # trial and test function for H(curl) X H(div)
    #uc, ud , = fes.TrialFunction()
    #vc, vd , = fes.TestFunction()


    print("Create the inverse of the submatrix M_g") 
    u,v = Hcc.TnT()
    gdofs = fes.Range(0)
    emb_g = Embedding(fes.ndof, gdofs)
    inv_g = BilinearForm(InnerProduct(u,v)*dx).Assemble().mat.Inverse(inverse="sparsecholesky")
    
    print("create the inverse of M_k")
    u,v = Hdd.TnT()
    kdofs = fes.Range(1)
    emb_k = Embedding(fes.ndof, kdofs)
    inv_k = BilinearForm(InnerProduct(u,v)*dx).Assemble().mat.Inverse(inverse="sparsecholesky")
    

    print("create the passage matrix M_kg : H(cc) -> H(dd)")
    u = Hcc.TrialFunction()
    v = Hdd.TestFunction()
    M_kg = BilinearForm(trialspace=Hcc, testspace=Hdd, geom_free = True)
    M_kg +=  InnerProduct(u, v)*dx
    M_kg.Assemble()


    print("create the passage matrix M_gk: H(dd) -> H(cc)")
    u = Hcc.TestFunction()
    v = Hdd.TrialFunction()
    M_gk = BilinearForm(testspace=Hcc, trialspace=Hdd, geom_free = True)
    M_gk +=  InnerProduct(u, v)*dx
    M_gk.Assemble()
    
    # creating the matrix for the operator curl-curl:
    u , v= Hcc.TnT()

    print("Create the elements for Face and Edje")
    tE = specialcf.tangential(3,True)
    t  = specialcf.tangential(3)
    n  = specialcf.normal(3)

    bbndtang  = specialcf.EdgeFaceTangentialVectors(3)
    tef1 = bbndtang[:,0]
    tef2 = bbndtang[:,1]

    nu1 = Cross(tef1,tE)
    nu2 = Cross(tef2,tE)

    n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
    P_n = OuterProduct(n,n)
    Q = Id(3) - P_n 

    print("create the discrete-distributional inc operator")
    K = BilinearForm(Hcc)
        # angle deficit
    K+= (u[tef1,nu1] - u[tef2,nu2])*v[tE,tE]*dx(element_vb=BBND)
        # facet term
    K+= 2*InnerProduct(TensorCross(OuterProduct(n,n), u.Operator("christoffel")*n), v)*dx(element_boundary=True)
        # volume term
    K+= InnerProduct(u.Operator("inc"),v)*dx
    K.Assemble()
    


    print("create the discrete-distributional J operator")
    # We need to create the Operatro for S^-1:
    u , v= Hdd.TnT()
    J = BilinearForm(Hdd)
    J += InnerProduct(u,v)*dx  
    J += -InnerProduct(Tr(u),Tr(v))/2*dx
    J.Assemble()



    print("create the composition of matrices UP")
    Mup = emb_g @ inv_g @ M_gk.mat @ inv_k @ J.mat  @ emb_k.T
    print("create the composition of matrices Down")
    Mdown = emb_k @ inv_k @ M_kg.mat @ inv_g  @ K.mat @ emb_g.T 
    #M =  Mdown
    #   

    print("Create the projection operator for the divergence free part")
    (u, p) , (v, q) = fes_projection.TnT()
    
    P = BilinearForm(fes_projection)
    P +=  InnerProduct(v,u) * dx
    P +=  div( v)*p * dx
    P +=  -InnerProduct(v*n - v[n,n]*n , p - (p*n)*n ) * dx(element_boundary = True)
    P +=  div( u)*q * dx
    P +=  -InnerProduct(u*n - u[n,n]*n , q - (q*n)*n ) * dx(element_boundary = True)
    P += -1e-9*p*q * dx

    F = BilinearForm(fes_projection)
    F +=  InnerProduct(v,u) * dx

    P.Assemble()
    F.Assemble()

    print("Create the projection operator for the projection part")
    invP = P.mat.Inverse(inverse="sparsecholesky")


    if draw_g_field :
        print("draw g")
        scene_g = Draw(gf_g, mesh,clipping={ "z" :-1 },draw_surf=False)
    if draw_divk_field :
        print("draw div k")
        scene_divd = Draw(div(gf_k), mesh,clipping={ "z" :-1 },draw_surf=False)
    if draw_k_field :
        print("draw k")
        scene_k = Draw(gf_k, mesh,clipping={ "z" :-1 })

    t =   final_time
    tau = step_time
    h = 0

    
    while h < t :
        
        h+=tau
        #we use only one matrix of the form described in the above markdowm
        #input("top part :")
        weak_field.vec.data += -tau * Mdown *weak_field.vec
        #input("top part :")
        weak_field.vec.data += tau * Mup *weak_field.vec
        #input("bot part : ")

        #elfield.vec.data += tau* (inv @ Mm ) @ inv * elfield.vec
        #scene_d.Redraw()
        
        Pk.vec.data = gf_k.vec
        projection_field.vec.data =  invP*(F.mat*projection_field.vec)
        gf_k.vec.data = Pk.vec
        
        if draw_g_field :
            scene_g.Redraw()
        if draw_divk_field :
            scene_divd.Redraw()
        if draw_k_field :
            scene_k.Redraw()
        print("time : " +str(h))

    return gf_g,gf_k

in the next function the space $H(dd)\times H(c)$ is treated as if it was a strees field. Therefore the component 

In [10]:


def EinsteinPropagation2(mesh , order = 1 , initail_g_field = None , bonus_intorder = 10 ,draw_g_field = False , draw_k_field = False, draw_divk_field = False , final_time  = 0.1, step_time = 0.002 ,video = False):
    

    print("Create FESpaces H(cc) , H(dd) and H(c)")
    Hcc = HCurlCurl(mesh, order=order)
    Hdd = HDivDiv(mesh, order=order   , dirichlet= ".*")
    Hc = HCurl(mesh,  order=order-1)

    print("Create projection FES H(dd)XH(c)")
    fes =  Hdd*Hc

    print(" initialize the g field")
    gf_g = GridFunction(Hcc)
    gf_g.Set ( initail_g_field, bonus_intorder=bonus_intorder, dual=True) #maybe without dual


    # projectional part
    print("Create stres field GridFunction for H(dd) X H(c)")
    stress_field = GridFunction(fes) 
    gf_k, gf_p = stress_field.components

    print("Create the inverse of the submatrix M_gg") 
    u,v = Hcc.TnT()
    M_gg = BilinearForm(InnerProduct(u,v)*dx).Assemble()
    inv_M_gg =M_gg.mat.Inverse(inverse="sparsecholesky")
    
    print("create the inverse of M_kk")
    u,v = Hdd.TnT()
    M_kk = BilinearForm(InnerProduct(u,v)*dx).Assemble()
    inv_M_kk = M_kk.mat.Inverse(inverse="sparsecholesky")

    # the galerkin method generates a matrix with each row a "test function" in the codomain space
    # and column corresponding to (a dof or) a "trial function" in the domain space
    # from the test function to a trial function!!!
    print("create the passage matrix M_gk : H(dd) -> H(cc)")
    u = Hcc.TrialFunction() 
    v = Hdd.TestFunction()
    M_kg = BilinearForm(trialspace=Hcc, testspace=Hdd, geom_free = True)
    M_kg += InnerProduct(u, v)*dx
    M_kg.Assemble()


    print("create the passage matrix M_kg: H(cc) -> H(dd)")
    u = Hcc.TestFunction()
    v = Hdd.TrialFunction()
    M_gk = BilinearForm(testspace=Hcc, trialspace=Hdd, geom_free = True)
    M_gk +=  InnerProduct(u, v)*dx
    M_gk.Assemble()
    
    # creating the matrix for the operator curl-curl:
    u , v= Hcc.TnT()

    print("Create the elements for Face and Edje")
    tE = specialcf.tangential(3,True)
    t  = specialcf.tangential(3)
    n  = specialcf.normal(3)

    bbndtang  = specialcf.EdgeFaceTangentialVectors(3)
    tef1 = bbndtang[:,0]
    tef2 = bbndtang[:,1]

    nu1 = Cross(tef1,tE)
    nu2 = Cross(tef2,tE)

    n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
    P_n = OuterProduct(n,n)
    Q = Id(3) - P_n 

    print("create the discrete-distributional inc operator")
    INC = BilinearForm(Hcc)
        # angle deficit
    INC+= (u[tef1,nu1] - u[tef2,nu2])*v[tE,tE]*dx(element_vb=BBND)
        # facet term
    INC+= 2*InnerProduct(TensorCross(OuterProduct(n,n), u.Operator("christoffel")*n), v)*dx(element_boundary=True)
        # volume term
    INC+= InnerProduct(u.Operator("inc"),v)*dx
    INC.Assemble()
    


    print("create the discrete-distributional J operator")
    # We need to create the Operatro for S^-1:
    u , v= Hdd.TnT()
    J = BilinearForm(Hdd)
    J += InnerProduct(u,v)*dx  
    J += -InnerProduct(Tr(u),Tr(v))/2*dx
    J.Assemble()

    print("Create the projection operator for the divergence free part")
    (u, p) , (v, q) = fes.TnT()
    
    PROJ = BilinearForm(fes)
    PROJ +=  InnerProduct(v,u) * dx
    PROJ +=  div( v)*p * dx
    PROJ +=  -InnerProduct(v*n - v[n,n]*n , p - (p*n)*n ) * dx(element_boundary = True)
    PROJ +=  div( u)*q * dx
    PROJ +=  -InnerProduct(u*n - u[n,n]*n , q - (q*n)*n ) * dx(element_boundary = True)
    PROJ += -1e-9*p*q * dx

    MASS = BilinearForm(fes)
    MASS +=  (InnerProduct(v,u)+InnerProduct(p,q)) * dx

    PROJ.Assemble()
    MASS.Assemble()

    print("Create the projection operator for the projection part")
    inv_PROJ = PROJ.mat.Inverse(inverse="sparsecholesky")


    if draw_g_field :
        print("draw g")
        scene_g = Draw(gf_g, mesh,clipping={ "z" :-1 },draw_surf=False)
    if draw_divk_field :
        print("draw div k")
        scene_divd = Draw(gf_p, mesh,clipping={ "z" :-1 },draw_surf=False)
    if draw_k_field :
        print("draw k")
        scene_k = Draw(gf_k, mesh,clipping={ "z" :-1 })



    print("create the composition of matrices that multiplies gf_k")
    Mk = inv_M_gg @ M_gk.mat @ inv_M_kk @ J.mat 
    print("create the composition of matrices that multiplies gf_g")
    Mg = inv_M_kk @ M_kg.mat @ inv_M_gg  @ INC.mat 
    print("other important stuff")
    gf_k_2 = stress_field.components[0]

    t =   final_time
    tau = step_time
    h = 0
    
    while h < t :
        
        h+=tau
        
        #we a simplectic euler integrator
        gf_k.vec.data += -tau * Mg * gf_g.vec
        gf_g.vec.data +=  tau * Mk * gf_k.vec
        
        gf_k_2.vec.data = gf_k.vec
        stress_field.vec.data =  inv_PROJ*(MASS.mat*stress_field.vec)
        
        if draw_g_field :
            scene_g.Redraw()
        if draw_divk_field :
            scene_divd.Redraw()
        if draw_k_field :
            scene_k.Redraw()
        print("time : " +str(h))

    return gf_g,gf_k


In [11]:
final_time = 0.1
step_time = 0.0005
video = False
draw_divk_field = True
draw_k_field = True
draw_g_field = True

maxh = 0.2
order = 2
shape = Box((-1,-1,-1), (1,1,1)) # create a box
geo = OCCGeometry(shape) # create a geometry from the box
mesh = Mesh(geo.GenerateMesh(maxh=maxh)) # create a mesh from the geometry


# define initial conditions for the wave
#peak = IfPos((1-(x-0.5)**2 -(y-0.5)**2 -(z-0.5)**2)**2,exp(25)*exp(-25/(1-(x-0.5)**2 -(y-0.5)**2 -(z-0.5)**2)) , 1)
peak = exp(-16*(x*x+y*y+z*z))
Draw(peak,mesh)#, clipping=(0,0,1,0))


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

BaseWebGuiScene

In [ ]:

PEAK = CF( (peak, 0, 0 ,0,0,0  , 0,0,0), dims=(3,3))
Draw(PEAK,mesh, clipping={ "z" :-1 })


In [ ]:

initail_g_field = PEAK#CF ( J(SymCurl(PEAK)), dims=(3,3))
Draw(initail_g_field,mesh, clipping={ "z" :-1 })


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

BaseWebGuiScene

In [ ]:

with TaskManager():
    g,k = EinsteinPropagation2(mesh , order = order ,initail_g_field = initail_g_field , final_time  = final_time, step_time = step_time, video =video , draw_divk_field = draw_divk_field , draw_k_field = draw_k_field , draw_g_field = draw_g_field )

Create FESpaces H(cc) , H(dd) and H(c)
Create projection FES H(dd)XH(c)
 initialize the g field
Create stres field GridFunction for H(dd) X H(c)
Create the inverse of the submatrix M_gg


KeyboardInterrupt: 